<a href="https://colab.research.google.com/github/DmitriySmurov/ControlWork1/blob/main/Seminar_1_Marketing_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Первое знакомство 😊

Чтобы загрузить исходные данные из CSV файла используем функционал Google Colab

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset.csv


Загружаем данные в датафрейм

In [ ]:
import io
import pandas as pd

df = pd.read_csv(io.BytesIO(uploaded['dataset(1).csv']))

df.head()

KeyError: 'dataset(1).csv'

Посмотрим основные характеристики датасета

# Посмотрим что внутри 🔎


In [ ]:
df.describe()

Приводим к нужному типу данных данных столбцы с датами и посмотрим временной диапазон данных

In [ ]:
df['event_date'] = pd.to_datetime(df['event_date'])
df['install_date'] = pd.to_datetime(df['install_date'])

print('min event date - ', df['event_date'].min())
print('max event date - ', df['event_date'].max())
print('min install date - ', df['install_date'].min())
print('max install date - ', df['install_date'].max())

NameError: name 'df' is not defined

# Вспомним, что такое когорты 😵

В качестве примера когортный отчет соберем для гео ▶ Russia

In [ ]:
df_installs = df[df['country_code'] == 'RU'].groupby('install_date', as_index=False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'installs'})
df_installs

NameError: name 'df' is not defined

# Мы готовы смотреть инсталлы ⛳

In [ ]:
import plotly.express as px

fig = px.line(df_installs, x = df_installs['install_date'], y = df_installs['installs'])
fig.show()

Т.к. судя по минимальному значению event date данные в датасете ограничены 1 апреля 2022 г. - ограничим этой датой также и инсталлы

In [ ]:
df_installs = df_installs[df_installs['install_date'] >= '2022-04-01']
df_installs

,install_date,installs
9,2022-04-01,4
10,2022-04-02,5
11,2022-04-03,5
12,2022-04-04,22
13,2022-04-05,6
...,...,...
126,2022-07-27,41
127,2022-07-28,47
128,2022-07-29,45
129,2022-07-30,55


Посмотрим на динамику инсталлов на скорректированных данных

In [ ]:
fig = px.bar(df_installs, x = 'install_date', y = 'installs')
fig.show()

Добавим атрибуцию в разрезе органика/закупка 🤼

In [ ]:
df_installs_upd = df[(df['install_date'] >= '2022-04-01') & (df['country_code'] == 'RU')].groupby(['install_date', 'is_organic'], as_index=False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'installs'})
df_installs_upd

,install_date,is_organic,installs
0,2022-04-01,True,4
1,2022-04-02,True,5
2,2022-04-03,True,5
3,2022-04-04,True,22
4,2022-04-05,True,6
...,...,...,...
120,2022-07-27,True,41
121,2022-07-28,True,47
122,2022-07-29,True,45
123,2022-07-30,True,55


In [ ]:
fig = px.line(df_installs_upd, x = df_installs_upd['install_date'], y = df_installs_upd['installs'], color=df_installs_upd['is_organic'])
fig.show()

Что можно сказать о закупке трафика из России? Попробуем посмотреть на трафик из США

In [ ]:
df_installs_us = df[(df['install_date'] >= '2022-04-01') & (df['country_code'] == 'US')].groupby(['install_date', 'is_organic'], as_index=False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'installs'})
df_installs_us

,install_date,is_organic,installs
0,2022-04-01,False,91
1,2022-04-01,True,42
2,2022-04-02,False,98
3,2022-04-02,True,55
4,2022-04-03,False,96
...,...,...,...
219,2022-07-27,True,277
220,2022-07-28,True,274
221,2022-07-29,True,330
222,2022-07-30,True,292


In [ ]:
fig_us = px.line(df_installs_us, x = df_installs_us['install_date'], y = df_installs_us['installs'], color=df_installs_us['is_organic'])
fig_us.show()

Какую еще информацию об инсталлах мы можем узнать из таблицы? Ваши варианты welcome ⛹

# Золотые люди 🤑

Узнаем число платящих в дневных когортах и конверсию в платящего. Сначала посчитаем сколько пользователей сконвертилось в платящего на каждый день

In [ ]:
df_payers = df[df['install_date'] >= '2022-04-01'].groupby(['appsflyer_id','install_date'], as_index = False).agg({'payments':'sum'})
df_payers = df_payers[df_payers['payments']>0]
df_payers = df_payers.groupby('install_date', as_index = False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'payers'})

df_payers

,install_date,payers
0,2022-04-01,19
1,2022-04-02,28
2,2022-04-03,21
3,2022-04-04,7
4,2022-04-05,4
...,...,...
117,2022-07-27,27
118,2022-07-28,30
119,2022-07-29,34
120,2022-07-30,40


In [ ]:
df_installs_all = df[df['install_date'] >= '2022-04-01'].groupby('install_date', as_index=False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'installs'})
df_cr = pd.merge(df_installs_all, df_payers, on='install_date', how='left')
df_cr['payers'] = df_cr['payers'].fillna(0)
df_cr['cr'] = round(df_cr['payers'] / df_cr['installs'], 2)

df_cr

,install_date,installs,payers,cr
0,2022-04-01,1010,19,0.02
1,2022-04-02,1024,28,0.03
2,2022-04-03,896,21,0.02
3,2022-04-04,311,7,0.02
4,2022-04-05,216,4,0.02
...,...,...,...,...
117,2022-07-27,3403,27,0.01
118,2022-07-28,3547,30,0.01
119,2022-07-29,4197,34,0.01
120,2022-07-30,4035,40,0.01


In [ ]:
fig_cr = px.line(df_cr, x='install_date', y='cr')
fig_cr.show()

Что еще хотим узнать о платящих? 💸

# Собираем когортный отчет 🧑 в разрезе дня по США

In [ ]:
df_installs_cohort = df[(df['install_date'] >= '2022-04-01') & (df['country_code'] == 'US')].groupby('install_date', as_index=False).agg({'appsflyer_id':'nunique', 'revenue_usd':'sum'}).rename(columns = {'appsflyer_id':'installs'})
df_payers_cohort = df[(df['install_date'] >= '2022-04-01') & (df['country_code'] == 'US')].groupby(['appsflyer_id','install_date'], as_index = False).agg({'payments':'sum'})
df_payers_cohort = df_payers_cohort[df_payers_cohort['payments']>0]
df_payers_cohort = df_payers_cohort.groupby('install_date', as_index = False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'payers'})

df_cohort = pd.merge(df_installs_cohort, df_payers_cohort, on='install_date', how='left')
df_cohort['payers'] = df_cohort['payers'].fillna(0)
df_cohort['cr'] = round(df_cohort['payers'] / df_cohort['installs'], 2)
df_cohort['rpi'] = round(df_cohort['revenue_usd'] / df_cohort['payers'], 2)

df_cohort

,install_date,installs,revenue_usd,payers,cr,rpi
0,2022-04-01,133,37.910000,6,0.05,6.32
1,2022-04-02,152,83.800000,11,0.07,7.62
2,2022-04-03,155,68.850000,8,0.05,8.61
3,2022-04-04,84,31.930000,4,0.05,7.98
4,2022-04-05,51,0.990000,1,0.02,0.99
...,...,...,...,...,...,...
117,2022-07-27,277,115.770000,9,0.03,12.86
118,2022-07-28,274,52.900000,5,0.02,10.58
119,2022-07-29,330,102.820000,11,0.03,9.35
120,2022-07-30,292,65.880655,9,0.03,7.32




```
# Выбран кодовый формат
```

# Домашнее задание 🥶

Расчитать и собрать когортный отчет c использованием Jupyter Notebook / Colab, результаты можно вывести также в Google Sheets.Рекомендуем когорты собирать в недельном разрезе, страны поделить на тиры (например, США, тир 1 = Европа и т.д., достаточно 3-4 тира).

- Installs
- Payers - число платящих
- Purchases - число покупок
- CR - процент платящих
- RPI - доход на одну установку
- Revenue - доход в абсолютном значении

Какие выводы можно сделать о когортах в игре? Дополните анализ несколькими визуализациями.

In [ ]:
# Функция для классификации стран по уровням
def classify_tier(country_code):
    if country_code == 'US':
        return 'Tier 1'
    elif country_code in ['GB', 'DE', 'FR', 'IT', 'ES', 'NL', 'SE', 'BE', 'AT', 'DK', 'FI', 'GR', 'IE', 'LU', 'NO', 'PT', 'CH']:
        return 'Tier 2'
    else:
        return 'Tier 3'

df = pd.read_csv(io.BytesIO(uploaded['dataset(1).csv']))

# Преобразование столбцов даты
df['event_date'] = pd.to_datetime(df['event_date'])
df['install_date'] = pd.to_datetime(df['install_date'])

# Применение функции classify_tier для классификации стран по уровням
df['tier'] = df['country_code'].apply(classify_tier)

# Извлечение install_week
df['install_week'] = df['install_date'].dt.to_period('W')

# Группировка данных по install_week и уровню для расчета требуемых показателей
cohort_data = (
    df[df['install_date'] >= '2022-04-01']
    .groupby(['install_week', 'tier'], as_index=False)
    .agg(
        installs=pd.NamedAgg(column='appsflyer_id', aggfunc='nunique'),
        revenue=pd.NamedAgg(column='revenue_usd', aggfunc='sum'),
        purchases=pd.NamedAgg(column='payments', aggfunc='sum')
    )
)

# Расчет плательщиков
df_payers = df[df['payments'] > 0].groupby(['install_week', 'tier'], as_index=False).agg(payers=pd.NamedAgg(column='appsflyer_id', aggfunc='nunique'))
cohort_data = pd.merge(cohort_data, df_payers, on=['install_week', 'tier'], how='left')
cohort_data['payers'].fillna(0, inplace=True)

# Расчет CR (коэффициента конверсии) и RPI (выручки за установку)
cohort_data['cr'] = (cohort_data['payers'] / cohort_data['installs']).round(2)
cohort_data['rpi'] = (cohort_data['revenue'] / cohort_data['installs']).round(2)

cohort_data.head()


,install_week,tier,installs,revenue,purchases,payers,cr,rpi
0,2022-03-28/2022-04-03,Tier 1,439,190.560000,44,34,0.08,0.43
1,2022-03-28/2022-04-03,Tier 2,110,39.188908,12,8,0.07,0.36
2,2022-03-28/2022-04-03,Tier 3,2385,323.763747,91,42,0.02,0.14
3,2022-04-04/2022-04-10,Tier 1,1067,478.890000,111,60,0.06,0.45
4,2022-04-04/2022-04-10,Tier 2,304,65.385337,12,6,0.02,0.22


In [ ]:
# Преобразование столбца install_week в строку
cohort_data['install_week'] = cohort_data['install_week'].astype(str)

# Построение графика количества установок по неделям и уровням
fig_installs = px.line(cohort_data, x='install_week', y='installs', color='tier', title='Количество установок в разбивке по неделям и уровням')
fig_installs.show()

# Построение графика выручки по неделям и уровням
fig_revenue = px.line(cohort_data, x='install_week', y='revenue', color='tier', title='Доход по неделям и уровням')
fig_revenue.show()



Домашнее задание:

In [ ]:
df_installs_cohort = df[(df['install_date'] >= '2022-04-01') & (df['country_code'] == 'UK' | df['country_code'] == 'FR' | df['country_code'] == 'DE')].groupby('install_date', as_index=False).agg({'appsflyer_id':'nunique', 'revenue_usd':'sum'}).rename(columns = {'appsflyer_id':'installs'})
df_payers_cohort = df[(df['install_date'] >= '2022-04-01') & (df['country_code'] == 'UK' | df['country_code'] == 'FR' | df['country_code'] == 'DE')].groupby(['appsflyer_id','install_date'], as_index = False).agg({'payments':'sum'})
df_payers_cohort = df_payers_cohort[df_payers_cohort['payments']>0]
df_payers_cohort = df_payers_cohort.groupby('install_date', as_index = False).agg({'appsflyer_id':'nunique', 'payments' : 'sum'}).rename(columns = {'appsflyer_id':'payers'})
df_cohort = pd.merge(df_installs_cohort, df_payers_cohort, on='install_date', how='left')
df_cohort['payers'] = df_cohort['payers'].fillna(0)
df_cohort['cr'] = round(df_cohort['payers'] / df_cohort['installs'], 2)
df_cohort['rpi'] = round(df_cohort['revenue_usd'] / df_cohort['payers'], 2)


df_cohort


TypeError: Cannot perform 'ror_' with a dtyped [object] array and scalar of type [bool]

In [ ]:
df_payers_cohort

,install_date,payers
0,2022-04-01,6
1,2022-04-02,11
2,2022-04-03,8
3,2022-04-04,4
4,2022-04-05,1
...,...,...
117,2022-07-27,9
118,2022-07-28,5
119,2022-07-29,11
120,2022-07-30,9
